In [1]:
from pymongo import MongoClient
from pprint import pprint
from random import randint
from bson.son import SON

In [2]:
client = MongoClient(port=27017)
client

MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True)

In [3]:
products_db = client['products']

In [4]:
products_db

Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'products')

In [5]:
product_collection = products_db['product_feedback']
product_collection

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'products'), 'product_feedback')

In [6]:
productlist = []
product_name = ['Mobile', 'TV', 'Washing machine', 'Refrigerator', 'Microwave Oven', 'Induction cooker', 'AC']
company_name = ['LG', 'Samsung', 'Bosch', 'Siemens', 'Whirlpool', 'Electrolux', 'Haier', 'Videocon']

In [7]:
for x in range(1, 1001):
    sale = {
        'name' : product_name[randint(0, (len(product_name)-1))],
        'rating' : randint(1, 5),
        'brand' : company_name[randint(0, (len(company_name)-1))] 
    }
    productlist.append(sale)

In [8]:
productlist[:5]

[{'name': 'Mobile', 'rating': 4, 'brand': 'Electrolux'},
 {'name': 'TV', 'rating': 1, 'brand': 'Siemens'},
 {'name': 'Induction cooker', 'rating': 1, 'brand': 'Electrolux'},
 {'name': 'Refrigerator', 'rating': 1, 'brand': 'Whirlpool'},
 {'name': 'Refrigerator', 'rating': 4, 'brand': 'Samsung'}]

In [9]:
product_collection.insert_many(productlist)

InsertManyResult([ObjectId('68209cf0a88a212cf10f5b76'), ObjectId('68209cf0a88a212cf10f5b77'), ObjectId('68209cf0a88a212cf10f5b78'), ObjectId('68209cf0a88a212cf10f5b79'), ObjectId('68209cf0a88a212cf10f5b7a'), ObjectId('68209cf0a88a212cf10f5b7b'), ObjectId('68209cf0a88a212cf10f5b7c'), ObjectId('68209cf0a88a212cf10f5b7d'), ObjectId('68209cf0a88a212cf10f5b7e'), ObjectId('68209cf0a88a212cf10f5b7f'), ObjectId('68209cf0a88a212cf10f5b80'), ObjectId('68209cf0a88a212cf10f5b81'), ObjectId('68209cf0a88a212cf10f5b82'), ObjectId('68209cf0a88a212cf10f5b83'), ObjectId('68209cf0a88a212cf10f5b84'), ObjectId('68209cf0a88a212cf10f5b85'), ObjectId('68209cf0a88a212cf10f5b86'), ObjectId('68209cf0a88a212cf10f5b87'), ObjectId('68209cf0a88a212cf10f5b88'), ObjectId('68209cf0a88a212cf10f5b89'), ObjectId('68209cf0a88a212cf10f5b8a'), ObjectId('68209cf0a88a212cf10f5b8b'), ObjectId('68209cf0a88a212cf10f5b8c'), ObjectId('68209cf0a88a212cf10f5b8d'), ObjectId('68209cf0a88a212cf10f5b8e'), ObjectId('68209cf0a88a212cf10f5b

Determine the number of products that have a rating of 5.

In [10]:
product_collection.count_documents({'rating': 5})

204

Determine the brands that have at least one product of rating 5.

In [11]:
product_collection.find({'rating': 5}).distinct('brand')

['Bosch',
 'Electrolux',
 'Haier',
 'LG',
 'Samsung',
 'Siemens',
 'Videocon',
 'Whirlpool']

Query to findout which brand has got maximum 5/5 rating we need to use aggregation pipeline

In [12]:
pipeline = [
    {
        '$match': {
            "rating": {"$in": [3, 4]}
        }
    },
    {
        "$group": {
            "_id": ["$brand", "$name", "$rating"],
            "count": {"$sum": 1}
        }
    },
    {
        "$sort": SON([("count", -1), 
                      ("_id", -1)])
    }
]

In [13]:
pprint(list(product_collection.aggregate(pipeline)))

[{'_id': ['Electrolux', 'Mobile', 4], 'count': 9},
 {'_id': ['Haier', 'AC', 4], 'count': 9},
 {'_id': ['Haier', 'Washing machine', 4], 'count': 8},
 {'_id': ['Videocon', 'Washing machine', 3], 'count': 8},
 {'_id': ['LG', 'AC', 3], 'count': 8},
 {'_id': ['Videocon', 'AC', 4], 'count': 7},
 {'_id': ['Haier', 'Microwave Oven', 3], 'count': 7},
 {'_id': ['Bosch', 'Induction cooker', 4], 'count': 7},
 {'_id': ['Electrolux', 'AC', 4], 'count': 7},
 {'_id': ['Whirlpool', 'AC', 3], 'count': 6},
 {'_id': ['Videocon', 'TV', 4], 'count': 6},
 {'_id': ['Videocon', 'Induction cooker', 3], 'count': 6},
 {'_id': ['LG', 'TV', 3], 'count': 6},
 {'_id': ['Samsung', 'Induction cooker', 4], 'count': 6},
 {'_id': ['Electrolux', 'Refrigerator', 4], 'count': 6},
 {'_id': ['Haier', 'Mobile', 4], 'count': 6},
 {'_id': ['Bosch', 'Mobile', 3], 'count': 6},
 {'_id': ['LG', 'Mobile', 4], 'count': 6},
 {'_id': ['Haier', 'Induction cooker', 4], 'count': 6},
 {'_id': ['Electrolux', 'Induction cooker', 3], 'count': 6

## Optional (Delete the data created.)

In [14]:
client.list_database_names()

['admin',
 'config',
 'local',
 'movielens',
 'pes',
 'products',
 'salesdb',
 'store']

In [15]:
products_db.drop_collection(name_or_collection='product_feedback')

{'nIndexesWas': 1, 'ns': 'products.product_feedback', 'ok': 1.0}

In [16]:
client.drop_database(name_or_database='products')

In [17]:
client.list_database_names()

['admin', 'config', 'local', 'movielens', 'pes', 'salesdb', 'store']